In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import schedule
import time
from sklearn.svm import SVR
from pandas.plotting import scatter_matrix
from pandas import DataFrame
from datetime import datetime
from sqlalchemy import create_engine
%matplotlib inline

In [2]:
#def etl():
    #Connect to MySQL
    engine_db = create_engine('mysql://root:@localhost/bitcoin1')
    engine_warehouse = create_engine('mysql://root:@localhost/bitwar')

    ####Extracting####
    #read CSV file
    df_raw = pd.read_csv("raw.csv")

    check = engine_db.has_table("raw_csv")
    if check == False:
        with engine_db.connect() as conn, conn.begin():
            df_raw.to_sql('raw_csv', conn, if_exists='append', index = False)

    #read existing data from db
    query = "select * from raw_csv"
    df_db = pd.read_sql_query(query,engine_db)

    #find new data if new csv or datasource provided
    df_diff = pd.concat([df_raw,df_db]).drop_duplicates(keep=False)    
    
    #Insert new data to DB
    with engine_db.connect() as conn, conn.begin():
        df_diff.to_sql('raw_csv', conn, if_exists='append', index = False)
    
    #####Transform####
    df_db_trans = pd.read_sql_query(query,engine_db)
    df_db_trans = df_db_trans.fillna(0) 

    #converting Timestamp to date and time
    df_db_trans['Timestamp'] = pd.to_datetime(df_db_trans.Timestamp, unit = 's')

    df_db_trans = df_db_trans.set_index('Timestamp')

    df_month = df_db_trans.resample('M').mean()
    df_month = df_month.dropna()

    df_year = df_db_trans.resample('Y').mean()
    df_year = df_year.dropna()

    df_daily = df_db_trans.resample('D').mean()
    df_daily = df_daily.dropna()

    ###LOADING###
    #Insert transformed data to warehouse
    with engine_warehouse.connect() as conn, conn.begin():
        df_month.to_sql('monthly', conn, if_exists='append', index = True)
        df_year.to_sql('yearly', conn, if_exists='append', index = True)
        df_daily.to_sql('daily', conn, if_exists='append', index = True)
    


    ###collecting data from warehouse for analysis
    monthly_query = "select * from monthly"
    yearly_query = "select * from yearly"
    daily_query = "select * from daily"

    df_monthly_from_warehouse =  pd.read_sql_query(monthly_query,engine_warehouse)
    df_yearly_from_warehouse =  pd.read_sql_query(yearly_query,engine_warehouse)
    df_daily_from_warehouse =  pd.read_sql_query(daily_query,engine_warehouse)
    print("done")



OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: NO)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [3]:
#def job():
 #   etl()
    
#schedule.every(1).minutes.do(job)
#while True:
  #  schedule.run_pending()
   # time.sleep(1)
    

done
done
done


KeyboardInterrupt: 